## Automated matching script

In [19]:
from openbabel import openbabel

In [20]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [22]:
def Canonical(array):
    Smiles = array[0]
    Canonical_smiles = Chem.CanonSmiles(Smiles)
    return(Canonical_smiles)

In [164]:
def split(word):
    return[char for char in word]

In [183]:
def inchi(array):
    #Smiles = array[0] #if no generations
    Smiles = array[1] # if generations
    mol = Chem.MolFromSmiles(Smiles)
    dummy = Chem.inchi.MolToInchiKey(mol)#split at first layer (contains everything but stereochemical info)
    characters = split(dummy)
    dummy = ''
    position = 0
    for i in range(len(characters)):
        if characters[i] == '-':
            position = i
            break
    for i in range(position):
        dummy += characters[i]
    return(dummy)

In [172]:
def parse(column):
    #for i in range(len(column)):
        #column[i] = column[i].replace('@','')
    call(f'sed "s/@//g" ./CHNO_Data/Prepped_CHNO_Smiles.tsv > test.tsv')
    return(column)

In [173]:
def prep_data(library_set, test_set): #input filepaths for the 2 sets as tsv files
    library_data = pd.read_csv(library_set, sep='\t')
    test_data = pd.read_csv(test_set, sep='\t')
    print(1)
    test_data['Inchi'] = test_data.apply(inchi, axis=1, raw=True, result_type='expand')
    print(2)
    library_data['Inchi'] = library_data.apply(inchi, axis=1, raw=True, result_type='expand')
    #library_data['Canonical Smiles'] = parse(library_data['Canonical Smiles'])
    #test_data['Canonical Smiles'] = parse(test_data['Canonical Smiles'])
    #print(3)
    return(library_data, test_data)

In [180]:
def find_matches(library_set, test_set):
    library_data, test_data = prep_data(library_set, test_set)
    #return (library_data, test_data)
    #print(library_data)
    #print(test_data)
    print(3)
    test_codes = []
    for i in range(len(test_data['Inchi'])):
        test_codes.append(test_data['Inchi'][i])
    library_codes = []
    for i in range(len(library_data['Inchi'])):
        library_codes.append(library_data['Inchi'][i])
    print(4)
    matches = []
    #for i in range(len(test_data['Inchi'])):
        #for j in range(len(library_data['Inchi'])):
            #if test_data['Inchi'][i] == library_data['Inchi'][j]:
                #matches.append(test_data['Inchi'][i])
                #print(i)
    for i in range(len(test_codes)):
        if test_codes[i] in library_codes:
            matches.append(test_codes[i])
    #print(matches)
    matches_smiles = []
    #matches_generations = []
    for i in range(len(matches)):
        for j in range(len(test_data['Inchi'])):
            if matches[i] == test_data['Inchi'][j]:
                matches_smiles.append(test_data['Smiles'][j])
              #  #matches_generations.append(test_data['Generation'][j])
    matches_data = {'Smiles': matches_smiles, 'Inchi': matches}
    library_test_matches = pd.DataFrame(matches_data)
    return(library_test_matches)

In [181]:
%%time
#!sed 's/@//g' ./CHO_Data/Prepped_CHO_Smiles.tsv > temp1.tsv
#!sed 's/@//g' ./UsingExampleData/formose_5_sep.tsv > temp2.tsv
a = find_matches('./CHO_Data/Prepped_CHO_Smiles.tsv', './UsingExampleData/formose_5_sep.tsv')

1
2
3
4
CPU times: user 10min 8s, sys: 5.6 s, total: 10min 13s
Wall time: 10min 16s


In [182]:
a

,Smiles,Inchi


In [178]:
a.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep='\t', mode='a') 